In [7]:
# Standard library imports
import os
import math
import glob
import json
import pickle

# Third-party library imports
import torch
import torchvision
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import pydicom
import sklearn
import sklearn.metrics

# Local module imports
import utils
import video_utils

In [8]:
device=torch.device("cuda")

In [9]:
vc_checkpoint = torch.load("model_data/weights/view_classifier.ckpt")
vc_state_dict={key[6:]:value for key,value in vc_checkpoint['state_dict'].items()}
view_classifier = torchvision.models.convnext_base()
view_classifier.classifier[-1] = torch.nn.Linear(
    view_classifier.classifier[-1].in_features, 11
)
view_classifier.load_state_dict(vc_state_dict)
view_classifier.to(device)
view_classifier.eval()
for param in view_classifier.parameters():
    param.requires_grad = False

/tmp/ipykernel_5393/2483250126.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vc_checkpoint = torch.load("model_data/weights/view_classifier.ckpt")


In [10]:
frames_to_take=32
frame_stride=2
video_size=224
mean = torch.tensor([29.110628, 28.076836, 29.096405]).reshape(3, 1, 1, 1)
std = torch.tensor([47.989223, 46.456997, 47.20083]).reshape(3, 1, 1, 1)

def process_dicoms(INPUT):
    """
    Reads DICOM video data from the specified folder and returns a tensor 
    formatted for input into the EchoPrime model.

    Args:
        INPUT (str): Path to the folder containing DICOM files.

    Returns:
        stack_of_videos (torch.Tensor): A float tensor of shape  (N, 3, 16, 224, 224)
                                        representing the video data where N is the number of videos,
                                        ready to be fed into EchoPrime.
    """

    dicom_paths = glob.glob(f'{INPUT}/**/*.dcm',recursive=True)
    stack_of_videos=[]
    for idx, dicom_path in tqdm(enumerate(dicom_paths),total=len(dicom_paths)):
        try:
            # simple dicom_processing
            dcm=pydicom.dcmread(dicom_path)
            pixels = dcm.pixel_array
            
            # exclude images like (600,800) or (600,800,3)
            if pixels.ndim < 3 or pixels.shape[2]==3:
                continue 
                
            # if single channel repeat to 3 channels    
            if pixels.ndim==3:
                
                pixels = np.repeat(pixels[..., None], 3, axis=3)
            
            # mask everything outside ultrasound region
            pixels=video_utils.mask_outside_ultrasound(pixels)
            
            
            
            #model specific preprocessing
            x = np.zeros((len(pixels),224,224,3))
            for i in range(len(x)):
                x[i] = video_utils.crop_and_scale(pixels[i])
            
            x = torch.as_tensor(x, dtype=torch.float).permute([3,0,1,2])
            # normalize
            x.sub_(mean).div_(std)
        
            ## if not enough frames add padding
            if x.shape[1] < frames_to_take:
                padding = torch.zeros(
                (
                    3,
                    frames_to_take - x.shape[1],
                    video_size,
                    video_size,
                ),
                dtype=torch.float,
                )
                x = torch.cat((x, padding), dim=1)
                
            start=0
            stack_of_videos.append(x[:, start : ( start + frames_to_take) : frame_stride, : , : ])
            
        except Exception as e:
            print("corrupt file")
            print(str(e))

    stack_of_videos=torch.stack(stack_of_videos)
    
    return stack_of_videos

In [11]:
def get_view_list(stack_of_videos, visualize=False, output=None):
    """
    Args:
        stack_of_videos (torch.Tensor): A float tensor with preprocessed echo video data
        
    Returns:
        view_list A list of predicted views
    """
    ## get views   
    stack_of_first_frames = stack_of_videos[:,:,0,:,:].to(device)
    with torch.no_grad():
        out_logits=view_classifier(stack_of_first_frames)
    out_views=torch.argmax(out_logits,dim=1)
    view_list = [utils.COARSE_VIEWS[v] for v in out_views]

    # visualize images and the assigned views
    if visualize:
        print("Preprocessed and normalized video inputs")
        rows, cols = (len(view_list) // 12 + (len(view_list) % 9 > 0)), 12
        fig, axes = plt.subplots(rows, cols, figsize=(cols, rows))
        axes = axes.flatten()
        for i in range(len(view_list)):
            display_image = (stack_of_first_frames[i].cpu().permute([1,2,0]) * 255).numpy()
            display_image = np.clip(display_image, 0, 255).astype('uint8')
            display_image = np.ascontiguousarray(display_image)
            display_image = cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR)
            cv2.putText(display_image, view_list[i].replace("_"," "), (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 220, 255), 2)
            if output:
                cv2.imwrite(os.path.join(output, f"{i}.jpg"), display_image)
            axes[i].imshow(display_image)
            axes[i].axis('off')

        for j in range(i + 1, len(axes)):
            axes[j].axis('off')
        plt.subplots_adjust(wspace=0.05, hspace=0.05)
        plt.show()

        
    return view_list

In [12]:
INPUT = "/media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/"
OUTPUT = "/media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/output"
os.makedirs(OUTPUT, exist_ok=True)

In [16]:
from deep_utils import DirUtils
patients = DirUtils.list_dir_full_path(INPUT, only_directories=True, dir_depth=2, exact_depth=True)
patients

['/media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/Andreas Jost-0760002100/TRANSTHORAKALES ECHO (TTE)-20240429-SYN1279758',
 '/media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/Andreas Von Känel-0008411786/TRANSTHORAKALES ECHO (TTE)-20240607-SYN1303838',
 '/media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/André Allemann-0015028658/TRANSTHORAKALES ECHO (TTE)-20240919-SYN1366377',
 '/media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/André Allemann-0015028658/kard TTE amb-20240305-KISIM11785085',
 '/media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/Bernd Krestan-0760215760/TRANSTHORAKALES ECHO (TTE)-20240819-SYN1345874',
 '/media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/Bernd Krestan-0760215760/TRANSTHORAKALES ECHO (TTE)-20240819-SYN1345874_output',
 '/media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/Christian Schmid-0003359050/TRANSTHORAKALES ECHO (TTE)-20240517-SYN12903

In [21]:
from os.path import split, join

for patient in patients:
    print(f"[INFO] Working on {patient}")
    patient_names = []
    p = patient
    for _ in range(2):
        p , name_ = split(p)
        patient_names.append(name_)
    patient_name = join(*patient_names[::-1])
    output_patient_path =  join(OUTPUT, patient_name)
    print(f"output: {output_patient_path}")
    os.makedirs(output_patient_path, exist_ok=True)
    numpy_path = join(output_patient_path, "numpy")
    
    if os.path.exists(numpy_path):
        stack_of_videos = np.load(numpy_path)
    else:
        stack_of_videos=process_dicoms(INPUT)
        np.save(numpy_path, stack_of_videos)
    
    print(f"stack_videos are extracted: {stack_of_videos.shape} for input: {patient}")
    
    view_list=get_view_list(stack_of_videos,visualize=True, output=output_patient_path)
    
    print(view_list)

[INFO] Working on /media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/Andreas Jost-0760002100/TRANSTHORAKALES ECHO (TTE)-20240429-SYN1279758
output: /media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/output/Andreas Jost-0760002100/TRANSTHORAKALES ECHO (TTE)-20240429-SYN1279758


  2%|▉                                       | 131/5767 [01:06<51:25,  1.83it/s]

corrupt file
Unable to convert the pixel data: one of Pixel Data, Float Pixel Data or Double Float Pixel Data must be present in the dataset


  2%|▉                                       | 140/5767 [01:10<48:18,  1.94it/s]

corrupt file
Unable to convert the pixel data: one of Pixel Data, Float Pixel Data or Double Float Pixel Data must be present in the dataset


...

100%|████████████████████████████████████▉| 5756/5767 [1:10:38<00:10,  1.09it/s]

corrupt file
Unable to convert the pixel data: one of Pixel Data, Float Pixel Data or Double Float Pixel Data must be present in the dataset


100%|█████████████████████████████████████| 5767/5767 [1:10:43<00:00,  1.36it/s]


stack_videos are extracted: torch.Size([3494, 3, 16, 224, 224]) for input: /media/aicvi/11111bdb-a0c7-4342-9791-36af7eb70fc0/Swiss-Care Echos/Andreas Jost-0760002100/TRANSTHORAKALES ECHO (TTE)-20240429-SYN1279758


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.90 GiB. GPU 0 has a total capacity of 23.53 GiB of which 9.87 GiB is free. Including non-PyTorch memory, this process has 13.21 GiB memory in use. Of the allocated memory 12.74 GiB is allocated by PyTorch, and 37.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)